In [2]:
import json
import sys
import time
import warnings
from functools import partial
from pathlib import Path
from typing import Literal

import lightning as L
import torch
from lightning.fabric.strategies import FSDPStrategy
from torch.distributed.fsdp.wrap import transformer_auto_wrap_policy

# support running without installing as a package
# wd = Path(__file__).parent.parent.resolve()
# sys.path.append(str(wd))

from generate.base import generate
from lit_parrot import Tokenizer
from lit_parrot.adapter import Block
from lit_parrot.adapter import Parrot, Config
from lit_parrot.adapter_v2 import add_adapter_v2_parameters_to_linear_layers
from lit_parrot.utils import lazy_load, check_valid_checkpoint_dir, quantization
from scripts.prepare_alpaca import generate_prompt



In [ ]:

def main(
    prompt: str = "What food do lamas eat?",
    input: str = "",
    adapter_path: Path = Path("out/adapter_v2/alpaca/lit_model_adapter_finetuned.pth"),
    checkpoint_dir: Path = Path(f"checkpoints/stabilityai/stablelm-base-alpha-3b"),
    quantize: Literal["llm.int8", "gptq.int4"] = None,
    max_new_tokens: int = 100,
    top_k: int = 200,
    temperature: float = 0.8,
    strategy: str = "auto",
    devices: int = 1,
    precision: str = "bf16-true",
) -> None:
    """Generates a response based on a given instruction and an optional input.
    This script will only work with checkpoints from the instruction-tuned Parrot-AdapterV2 model.
    See `finetune/adapter_v2.py`.

    Args:
        prompt: The prompt/instruction (Alpaca style).
        adapter_path: Path to the checkpoint with trained adapter weights, which are the output of
            `finetune/adapter_v2.py`.
        checkpoint_dir: The path to the checkpoint folder with pretrained Parrot weights.
        input: Optional input (Alpaca style).
        quantize: Whether to quantize the model and using which method:
            ``"llm.int8"``: LLM.int8() mode,
            ``"gptq.int4"``: GPTQ 4-bit mode.
        max_new_tokens: The number of generation steps to take.
        top_k: The number of top most probable tokens to consider in the sampling process.
        temperature: A value controlling the randomness of the sampling process. Higher values result in more random
            samples.
        strategy: Indicates the Fabric strategy setting to use.
        devices: How many devices to use.
        precision: Indicates the Fabric precision setting to use.
    """
    if strategy == "fsdp":
        auto_wrap_policy = partial(transformer_auto_wrap_policy, transformer_layer_cls={Block})
        strategy = FSDPStrategy(auto_wrap_policy=auto_wrap_policy, cpu_offload=False)


In [4]:
from pathlib import Path

checkpoint_dir = Path("checkpoints/tiiuae/falcon-7b")
adapter_path = Path("out/adapter/alpaca/lit_model_adapter_finetuned.pth")

quantize: Literal["llm.int8", "gptq.int4"] = None
max_new_tokens: int = 100
top_k: int = 200
temperature: float = 0.8
strategy: str = "auto"
devices: int = 1
precision: str = "bf16-true"

fabric = L.Fabric(devices=devices, precision=precision, strategy=strategy)
fabric.launch()

check_valid_checkpoint_dir(checkpoint_dir)

with open(checkpoint_dir / "lit_config.json") as fp:
    config = Config(**json.load(fp))

if quantize is not None and devices > 1:
    raise NotImplementedError
if quantize == "gptq.int4":
    model_file = "lit_model_gptq.4bit.pth"
    if not (checkpoint_dir / model_file).is_file():
        raise ValueError("Please run `python quantize/gptq.py` first")
else:
    model_file = "lit_model.pth"

checkpoint_path = checkpoint_dir / model_file

fabric.print(f"Loading model {str(checkpoint_path)!r} with {config.__dict__}", file=sys.stderr)
t0 = time.time()
with fabric.init_module(empty_init=True), quantization(quantize):
    model = Parrot(config)
    add_adapter_v2_parameters_to_linear_layers(model)
fabric.print(f"Time to instantiate model: {time.time() - t0:.02f} seconds.", file=sys.stderr)

t0 = time.time()
with lazy_load(checkpoint_path) as checkpoint, lazy_load(adapter_path) as adapter_checkpoint:
    checkpoint.update(adapter_checkpoint)
    model.load_state_dict(checkpoint, strict=quantize is None)
fabric.print(f"Time to load the model weights: {time.time() - t0:.02f} seconds.", file=sys.stderr)



You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, 'adapter_prompt_length': 10, 'adapter_start_layer': 2}
Time to instantiate model: 1.45 seconds.
Time to load the model weights: 33.09 seconds.


In [5]:
data_dir = Path("data/alpaca")

train_data = torch.load(data_dir / "train.pt")
val_data = torch.load(data_dir / "test.pt")
tokenizer = Tokenizer(checkpoint_dir / "tokenizer.json", checkpoint_dir / "tokenizer_config.json")



In [10]:
micro_batch_size = 1
max_seq_length = 800

In [11]:
def get_batch(fabric: L.Fabric, data: list, max_seq_length: int, deterministic: bool, iter_num: int = 0):
    if deterministic:
        ix = torch.arange(iter_num*micro_batch_size, (iter_num + 1)*micro_batch_size)
    else:
        ix = torch.randint(len(data), (micro_batch_size,))

    input_ids = [data[i]["input_ids"].type(torch.int64) for i in ix]
    labels = [data[i]["labels"].type(torch.int64) for i in ix]


    max_len = max(len(s) for s in input_ids) if fabric.device.type != "xla" else max_seq_length
    def pad_right(x, pad_id):
        # pad right based on the longest sequence
        n = max_len - len(x)
        return torch.cat((x, torch.full((n,), pad_id, dtype=x.dtype)))

    x = torch.stack([pad_right(x, pad_id=0) for x in input_ids])
    y = torch.stack([pad_right(x, pad_id=-1) for x in labels])

    if fabric.device.type in ("mps", "xla"):
        x, y = fabric.to_device((x, y))
    else:
        x, y = fabric.to_device((x.pin_memory(), y.pin_memory()))

    return x, y

In [13]:
def loss_fn(logits, targets):
    # shift the targets such that output n predicts token n+1
    logits = logits[..., :-1, :].contiguous()
    targets = targets[..., 1:].contiguous()
    loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
    return loss

0.87109375

In [46]:
import tqdm
data = val_data
losses = torch.zeros(len(data))
data_and_loss = []
for k in tqdm.trange(len(data)):
    input_ids, targets = get_batch(fabric, data, max_seq_length, True, k)
    logits = model(input_ids)
    loss = loss_fn(logits, targets)
    losses[k] = loss.item()
    data_and_loss.append(
        {
            "data": tokenizer.processor.decode(input_ids.tolist()[0]),
            "loss": losses[k].item()
        }
    )


100%|██████████| 2000/2000 [01:33<00:00, 21.41it/s]


In [48]:
import pandas as pd
df = pd.DataFrame(data_and_loss)
df.to_csv("neg_mining/06_20_val_out_v1.csv")

In [53]:
df.sort_values(by = 'loss').tail(n=30).values

array([['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a snippet that allows the user to input a first name with a width of 8 columns and inserts it into a sentence stating that the person cannot participate in grade-appropriate writing assignments in English.\n\n### Response:{formtext: name=first name; cols=8} cannot participate meaningfully in grade appropriate writing assignments in English.',
        2.28125],
       ['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a snippet that generates an apology email for delayed response. The snippet should include a placeholder for the expected timeframe for processing the request. Use appropriate TextBlaze templates to create the placeholders.\n\n### Response:Please accept our apologies for the delay in replying to your emails. Our backend systems went down and as a re

In [15]:
tokenizer.decode(input_ids)

TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
model.eval()
model = fabric.setup(model)

tokenizer = Tokenizer(checkpoint_dir / "tokenizer.json", checkpoint_dir / "tokenizer_config.json")
sample = {"instruction": prompt, "input": input}
prompt = generate_prompt(sample)
encoded = tokenizer.encode(prompt, device=model.device)
prompt_length = encoded.size(0)
max_returned_tokens = prompt_length + max_new_tokens

t0 = time.perf_counter()
y = generate(
    model,
    encoded,
    max_returned_tokens,
    max_seq_length=max_returned_tokens,
    temperature=temperature,
    top_k=top_k,
    eos_id=tokenizer.eos_id,
)
t = time.perf_counter() - t0

model.reset_cache()
output = tokenizer.decode(y)
output = output.split("### Response:")[1].strip()
fabric.print(output)

tokens_generated = y.size(0) - prompt_length
fabric.print(f"\n\nTime for inference: {t:.02f} sec total, {tokens_generated / t:.02f} tokens/sec", file=sys.stderr)
if fabric.device.type == "cuda":
    fabric.print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB", file=sys.stderr)


